In [1]:
import spacy
import math
from random import randint
from tqdm import tqdm
nlp = spacy.load('ja_ginza')

In [2]:
mask_ratio = 0.15
span_length = 3

In [3]:
input_list = []
target_list = []
sentences = []
for line in open('/home/chen/T5_mask_infilling/clinic_corpus.txt', 'r'):
    if line is not None:
        sentences.append(line.strip())
        

In [4]:
len(sentences)

11832

In [7]:
for i in tqdm(range(len(sentences))):    
    
    doc = nlp(sentences[i])
    mask_token = math.ceil(len(doc)*mask_ratio)
    chunk_num = math.ceil(mask_token/span_length)
    
    assert chunk_num != 0
    
    index_sequence = [i for i in range(len(doc)-span_length+1)]
    for i in range(len(index_sequence)):
        index_sequence[i] = [i for i in range(i,i+span_length)] 
        
    mask_index = []
    while chunk_num > 0:
        index_sequence_length = len(index_sequence)
        masked_spans = index_sequence[randint(0,index_sequence_length-1)]
        mask_index.extend(masked_spans)
        index_id = index_sequence.index(masked_spans)
        del index_sequence[index_id-span_length+1:index_id+span_length]
        chunk_num = chunk_num - 1    
        
    mask_index.sort()
    
    text_sequence = [i for i in range(len(doc))]
    unmask_index = list(set(text_sequence) - set(mask_index))
    
    unmask_index.sort()
    
    input_text = [None] * len(doc)
    for i in unmask_index:
        input_text[i] = doc[i]
        
        
    re_input_text = []
    for i in range(len(input_text)-1):
        if str(input_text[i]) == 'None' and str(input_text[i+1]) == 'None':
            pass
        else:
            re_input_text.append(input_text[i])
        i+=2               

    re_input_text.append(input_text[len(input_text)-1])   
    
    sent_token_id = 0
    for i in range(len(re_input_text)):
        if str(re_input_text[i]) == 'None':
            re_input_text[i] = f'<extra_id_{sent_token_id}>'
            sent_token_id += 1  
            
    inputs = ''.join(str(e) for e in re_input_text)    
    
    target_text = [None] * len(doc)
    for i in mask_index:
        target_text[i] = doc[i]
        
        
    re_target_text = []
    for i in range(len(target_text)-1):
        if str(target_text[i]) == 'None' and str(target_text[i+1]) == 'None':
            pass
        else:
            re_target_text.append(target_text[i])
        i+=2               

    re_target_text.append(target_text[len(target_text)-1])    
    
    sent_token_id = 0
    for i in range(len(re_target_text)):
        if str(re_target_text[i]) == 'None':
            re_target_text[i] = f'<extra_id_{sent_token_id}>'
            sent_token_id += 1 
            
    targets = ''.join(str(e) for e in re_target_text)  
    
    input_list.append(inputs)
    target_list.append(targets)

100%|█████████████████████████████████████| 11832/11832 [03:59<00:00, 49.38it/s]


In [8]:
assert len(input_list) == len(target_list)

In [9]:
input_list[500]

'胸腔ドレーンは徐々に浅くし,場合によりネラトンカテーテルに入れ替え菌の陰性化を確認し<extra_id_0>'

In [10]:
target_list[500]

'<extra_id_0>てから抜去した.'

In [11]:
input_list[1000]

'保存療法の効果が乏しい場合や疼痛<extra_id_0>,骨<extra_id_1>が激しい場合には手術療法も選択されている.'

In [12]:
target_list[1000]

'<extra_id_0>が強い場合<extra_id_1>破壊,変形<extra_id_2>'

In [13]:
file_out = '/home/chen/T5_mask_infilling/masked_clinic_corpus.txt'
with open(file_out, 'w', encoding = 'utf-8') as f:
    for i in range(len(input_list)):
        f.write(input_list[i].strip() + '\t' +target_list[i].strip() + '\n')
     
        
f.close() 